
## 1.3 Applying PyDP to famous ML dataset

The PyDP package provides a Python API into [Google's Differential Privacy library](https://github.com/google/differential-privacy). This example uses the alpha 0.1.7 version of the package that has the following limitations:



The sinking of the Titanic is one of the most infamous shipwrecks in history.

On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew.

While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.



### [Titanic Dataset Challenge](https://www.kaggle.com/c/titanic/overview):

We ask you to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (i.e. name, age, gender, socio-economic class, etc.).

### Bias in 2020

Its 2020 and we have realized that there are inherent bias in our beliefs and system and moving forward, we need to remove these bias. To solve this, we decided, why not to formulate the most common ML dataset in a more responsible manner. Can we use the technology at hand to solve this?

### Enter Differential Privacy

Using DP, we aim to reduce such bias in the system and we want all passengers to give an equal chance of survival. 

Consider a scenario where there is a ship near the accident site but depending on the demographic of people in Titanic, it would decide whether it will do a rescue operation or not. 

Now you have the passenger Manifesto in your hand the nearby ship can ask you few statistical questions before it can make a decision. So it as
now the passengers of Titanic decide that they will use DP to answer the following questions:


In [ ]:
# Install the PyDP package
! pip install python-dp

In [ ]:
import pydp as dp  # by convention our package is to be imported as dp (for Differential Privacy!)
from pydp.algorithms.laplacian import (
    BoundedSum,
    BoundedMean,
    BoundedStandardDeviation,
    Count,
    Max,
    Min,
    Median,
)
import pandas as pd
import statistics  # for calculating mean without applying differential privacy

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# get the cleaned dataset from our public github repo
url = "https://raw.githubusercontent.com/OpenMined/PyDP/dev/examples/Tutorial_3-Titanic_demo/titanic_clean.csv"
df = pd.read_csv(url, sep=",", index_col=0)
df.head()

### Q1. What is the average age of passengers:

In [ ]:
# calculates passenger age mean without applying differential privacy
def mean_age() -> float:
    return statistics.mean(list(df["Age"]))

In [ ]:
# calculates passenger age mean applying differential privacy
def private_mean(privacy_budget: float) -> float:
    x = BoundedMean(privacy_budget, lower_bound=0.1, upper_bound=90, dtype="float")
    return x.quick_result(list(df["Age"]))

In [ ]:
print("Mean: ", mean_age())
print("Private Mean: ", private_mean(0.8))

### Q2: How many Total passengners are on this ship?

In [ ]:
# Function to calculate total number of passengers without applying differential privacy.
def sum_passengers() -> int:
    return df.count()["Age"]

In [ ]:
# Function to calculate total number of passengers applying differential privacy.
def private_sum(privacy_budget: float) -> int:
    x = Count(privacy_budget, dtype="float")
    # return x.quick_result(list(df["Age"]))
    return x.quick_result(list(df["Age"]))

In [ ]:
print("Total:\t" + str(sum_passengers()))
print("Private Total:\t" + str(private_sum(1)))

### Q3: What is the age of the youngest passenger on the ship?

In [ ]:
# Function to return the minimum of the passengers age without appyling differential privacy.
def min_age() -> int:
    return df.min()[0]

In [ ]:
# Function to return the minimum of the passengers age appyling differential privacy.
def private_min(privacy_budget: float) -> float:
    # 0 and 150 are the upper and lower limits for the search bound.
    x = Min(privacy_budget, lower_bound=0.1, upper_bound=90, dtype="float")
    return x.quick_result(list(df["Age"]))

In [ ]:
print("Min:\t" + str(min_age()))
print("Private Min:\t" + str(private_min(1)))

### Q4: What is the age of the oldest passenger on the ship?

In [ ]:
# Function to return the minimum of the passengers age without appyling differential privacy.
def max_age() -> int:
    return df.max()[0]

In [ ]:
# Function to return the minimum of the passengers age appyling differential privacy.
def private_max(privacy_budget: float) -> float:
    # 0 and 150 are the upper and lower limits for the search bound.
    x = Max(privacy_budget, lower_bound=0.1, upper_bound=90, dtype="float")
    return x.quick_result(list(df["Age"]))

In [ ]:
print("Max:\t" + str(max_age()))
print("Private Max:\t" + str(private_max(1)))

### Q5: How many passengers are older than a certain age?

In [ ]:
# Calculates number of passengers which age is more than "limit" age without applying differential privacy.
def count_above(limit: int) -> int:
    return df[df.Age > limit].count()[0]

In [ ]:
# Calculates number of passengers which age is more than "limit" age applying differential privacy.
def private_count_above(privacy_budget: float, limit: int) -> int:
    x = Count(privacy_budget, dtype="float")
    return x.quick_result(list(df[df.Age > limit]["Age"]))

In [ ]:
print("Above 70:\t" + str(count_above(70)))
print("private count above:\t" + str(private_count_above(1, 70)))

### Q6: How many passengers are younger than a certain age?

In [ ]:
# Calculates number of passengers which age is less than "limit" age without applying differential privacy.
def count_below(limit: int) -> int:
    return df[df.Age < limit].count()[0]

In [ ]:
# Calculates number of passengers which age is less than "limit" age applying differential privacy.
def private_count_below(privacy_budget: float, limit: int) -> int:
    x = Count(privacy_budget, dtype="float")
    return x.quick_result(list(df[df.Age < limit]["Age"]))

In [ ]:
print("Below 21:\t" + str(count_below(21)))
print("private count below:\t" + str(private_count_below(1, 21)))

### Q7: What is the average fare of the passengers?

Consider the scenario that after the Tradegy, you are doing the data analysis for the local newspaper on the people who have survived. We divide the data into two bins, survived and not-survived. 

In the surived subset:

In [ ]:
# Total number of passengers
df.Survived.count()

In [ ]:
# Total number of passengers who survived
np.sum(df["Survived"] == 1)

In [ ]:
# Total number of passengers who didn't survive
np.sum(df["Survived"] == 0)

In [ ]:
sns.countplot(x="Survived", data=df, palette="hls", hue="Survived")
plt.xticks(rotation=45)
plt.show()

In [ ]:
sns.barplot(
    x="Survived", y="Fare", data=df, estimator=np.mean, ci=None, palette="Blues_d"
)
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Create the booleans mask for the two subsets
Not_survived_passengers = df["Survived"] == 0
Survived_passengers = df["Survived"] == 1

### Q8: we want to find the mean of the fare they paid




In [ ]:
# Calculates passenger age mean without applying differential privacy
def mean_fare() -> float:
    return df.groupby("Survived")["Fare"].mean()

In [ ]:
# Calculates passenger age mean applying differential privacy
def private_mean_s_fare(privacy_budget: float) -> float:
    x = BoundedMean(privacy_budget, lower_bound=0, upper_bound=600, dtype="float")
    return x.quick_result(list(df.loc[Not_survived_passengers, "Fare"]))

In [ ]:
# Calculates passenger age mean applying differential privacy
def private_mean_ns_fare(privacy_budget: float) -> float:
    x = BoundedMean(privacy_budget, lower_bound=0, upper_bound=600, dtype="float")
    return x.quick_result(list(df.loc[Survived_passengers, "Fare"]))

In [ ]:
# Without DP
print("Mean: ", mean_fare())

# With DP
print("Private Mean Not Survivors: ", private_mean_s_fare(0.8))
print("Private Mean Survivors: ", private_mean_ns_fare(0.8))

### Q9: we want to find the median of fare they paid


In [ ]:
# Calculates passenger age mean without applying differential privacy
def median_fare() -> float:
    return df.groupby("Survived")["Fare"].median()

In [ ]:
# Calculates passenger age mean applying differential privacy
def private_median_s_fare(privacy_budget: float) -> float:
    x = Median(privacy_budget, lower_bound=0, upper_bound=600, dtype="float")
    return x.quick_result(list(df.loc[Not_survived_passengers, "Fare"]))

In [ ]:
# Calculates passenger age mean applying differential privacy
def private_median_ns_fare(privacy_budget: float) -> float:
    x = Median(privacy_budget,lower_bound=0, upper_bound=600, dtype="float")
    return x.quick_result(list(df.loc[Survived_passengers, "Fare"]))

In [ ]:
# Without DP
print("Median Deviation: ", median_fare())

# With DP
print("Private Median Not Survivors: ", private_median_s_fare(0.8))
print("Private Median Survivors: ", private_median_ns_fare(0.8))

### Q10: we want to find the deviation of fare they paid

In [ ]:
# Calculates passenger age mean without applying differential privacy
def std_fare() -> float:
    return df.groupby("Survived")["Fare"].std()

In [ ]:
# Calculates passenger age mean applying differential privacy
def private_std_s_fare(privacy_budget: float) -> float:
    x = BoundedStandardDeviation(privacy_budget, lower_bound=0, upper_bound=600, dtype="float")
    return x.quick_result(list(df.loc[Not_survived_passengers, "Fare"]))

In [ ]:
# Calculates passenger age mean applying differential privacy
def private_std_ns_fare(privacy_budget: float) -> float:
    x = BoundedStandardDeviation(privacy_budget,lower_bound=0, upper_bound=600, dtype="float")
    return x.quick_result(list(df.loc[Survived_passengers, "Fare"]))

In [ ]:
# Without DP
print("Standard Deviation: ", std_fare())

# With DP
print("Private Std Not Survivors: ", private_std_s_fare(0.8))
print("Private Std Survivors: ", private_std_ns_fare(0.8))